# EEG - Flow

## 2. Select bad channels and annotate bad segments

The bad channels and the bad segments must be annotated and excluded from subsequent analysis. 

Last edit: 06.05.2023 00:29
@anguyen

In [ ]:
import os
from itertools import chain

from mne.io import read_raw_fif, write_info
from mne.preprocessing import (
    compute_bridged_electrodes,
    interpolate_bridged_electrodes
)
from mne.viz import set_browser_backend
from pyprep import NoisyChannels

from eeg_flow.config import load_config
from eeg_flow.utils.annotations import merge_bad_annotations
from eeg_flow.utils.bids import get_fname, get_folder
from eeg_flow.utils.concurrency import lock_files
from eeg_flow.viz import plot_bridged_electrodes

%matplotlib qt
set_browser_backend('qt')

_, DERIVATIVES_FOLDER_ROOT, _ = load_config()

The parameters of the file to process are defined below. Locks are created to prevent someone else from running the same task and from writing the same derivatives.

In [ ]:
PARTICIPANT = "P02" 
GROUP       = "G2"
TASK        = "oddball"
RUN         = 1
"""
PARTICIPANT: str (for example "P03", "P28")
GROUP:       str ["G1", "G2", "G3", "G4", "G5", "G6", "G7", "G8"]
TASK:        str, ["oddball", "UT"]
RUN:         int [1,2]
"""

FNAME_STEM = get_fname(PARTICIPANT, GROUP, TASK, RUN)
#DERIVATIVES_FOLDER = get_folder(DERIVATIVES_FOLDER_ROOT, PARTICIPANT, GROUP)
#DERIVATIVES_SUBFOLDER = DERIVATIVES_FOLDER / FNAME_STEM
DERIVATIVES_SUBFOLDER = get_folder(
    DERIVATIVES_FOLDER_ROOT, PARTICIPANT, GROUP, TASK, RUN
)

# create derivatives plots subfolder
os.makedirs(DERIVATIVES_SUBFOLDER / "plots", exist_ok=True)

# create locks
derivatives = (
    DERIVATIVES_SUBFOLDER / (FNAME_STEM + "_step2_info.fif"),
    DERIVATIVES_SUBFOLDER / (FNAME_STEM + "_step2_oddball_with_bads_annot.fif"),
    DERIVATIVES_SUBFOLDER / "plots" / (FNAME_STEM + "_step2_bridges.svg"),
)
locks = lock_files(*derivatives)

# load raw recording
raw = read_raw_fif(
    DERIVATIVES_SUBFOLDER / (FNAME_STEM + "_step1_raw.fif"), preload=True
)

## 2.1 Bridges

More information can be found on this [MNE tutorial](https://mne.tools/stable/auto_examples/preprocessing/eeg_bridging.html).

In [ ]:
FNAME_BRIDGE_PLOT = (
    DERIVATIVES_SUBFOLDER / "plots" / (FNAME_STEM + "_step2_bridges.svg")
)

if not FNAME_BRIDGE_PLOT.exists():
    fig, ax = plot_bridged_electrodes(raw)
    fig.suptitle(FNAME_STEM, fontsize=16, y=1.0)
    fig.savefig(FNAME_BRIDGE_PLOT, transparent=True)

If the bridges don't look fixable, the recording should be probably dropped. Else, they are interpolated:

In [ ]:
raw.set_montage("standard_1020")  # we need a montage for the interpolation
bridged_idx, _ = compute_bridged_electrodes(raw)
try:
    raw = interpolate_bridged_electrodes(raw, bridged_idx)
except RuntimeError:
    bads_idx = sorted(set(chain(*bridged_idx)))
    raw.info["bads"] = [raw.ch_names[k] for k in bads_idx]
    assert "M1" not in raw.info["bads"]
    assert "M2" not in raw.info["bads"]

## 2.2 Bad channels

Bad channels are suggested by `pyprep`. A visual inspection is however needed to confirm or modify the output from `pyprep`.

In [ ]:
%%time
raw.filter(
    l_freq=1.0,
    h_freq=100.0,
    picks="eeg",
    method="fir",
    phase="zero-double",
    fir_window="hamming",
    fir_design="firwin",
    pad="edge",
)

ns = NoisyChannels(raw, do_detrend=False)  # operates only on EEG
ns.find_bad_by_SNR()
ns.find_bad_by_correlation()
ns.find_bad_by_hfnoise()
ns.find_bad_by_nan_flat()
# ns.find_bad_by_ransac()  # requires electrode position
print (ns.get_bads())

raw.info["bads"].extend(
    [ch for ch in ns.get_bads() if ch not in ("M1", "M2")]
)
raw.info["bads"] = list(set(raw.info["bads"]))

## 2.3 Visual inspection and annotations of bad segments

During the visual inspection, bad segments should be annotated.

In [ ]:
raw.filter(
    l_freq=1.0,
    h_freq=40.0,
    picks="eeg",
    method="fir",
    phase="zero-double",
    fir_window="hamming",
    fir_design="firwin",
    pad="edge",
)

In [ ]:
raw.plot(theme="light")

In [ ]:
raw.annotations

deprecated (The segments annotated as bads are mapped onto the oddball events, to standardize the annotation procedure and to mark only the signal used in the `ICA` and in the `Epochs` as bad.)

In [ ]:
annotations = raw.annotations

## 2.4 Save derivatives

The updated annotations can now be saved alongside the selected bad channels.

In [ ]:
FNAME_INFO = DERIVATIVES_SUBFOLDER / (FNAME_STEM + "_step2_info.fif")
assert not FNAME_INFO.exists()  # write_info always overwrites
write_info(FNAME_INFO, raw.info)
FNAME_ANNOT = (
    DERIVATIVES_SUBFOLDER / (FNAME_STEM + "_step2_oddball_with_bads_annot.fif")
)
annotations.save(FNAME_ANNOT, overwrite=False)

Regardless of the success of the task, the locks must be released.
If this step is forgotten, someone might have to remove the corresponding `.lock` file manually.

In [ ]:
for lock in locks:
    lock.release()
del locks  # delete would release anyway